In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

## Template matching y descriptores
Objetivos:
1. Obtener una detección del logo en cada imagen sin falsos positivos
2. Plantear y validar un algoritmo para múltiples detecciones en la imagen `coca_multi.png` con el mismo template del ítem 1
3. Generalizar el algoritmo del ítem 2 para todas las imágenes

Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza de la detección.

### Template e imagenes a analizar

In [ ]:
TEMPLATE_PATH = "./template/pattern.png"
IMAGES_PATH = "./images/"

template_color = cv2.imread(TEMPLATE_PATH, cv2.IMREAD_COLOR)
images_color = [
    cv2.imread(IMAGES_PATH + image_name, cv2.IMREAD_COLOR)
    for image_name in os.listdir(path=IMAGES_PATH)
    if not "multi" in image_name
]

### Pre-procesamiento de imagenes
<!-- - Se normaliza la imagen mediante una ecualización del histograma -->
- Conversión a escala de grises
- Aplicación de Gaussian Blue para reducción de ruido
- Detección de límites con Canny

In [ ]:
def preprocess_image(input_image: np.ndarray) -> np.ndarray:
    """Encapsulates the pre-processing on images"""
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 1)
    # edges_image = cv2.Canny(blurred_image, threshold1=50, threshold2=150)
    return blurred_image

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3)
fig.set_size_inches(15, 5)
template1 = preprocess_image(template_color)
template2 = -preprocess_image(template_color)
axs[0].imshow(cv2.cvtColor(template_color, cv2.COLOR_BGR2RGB))
axs[1].imshow(template1, cmap="gray")
axs[2].imshow(template2, cmap="gray")
axs[0].set_title("Original template")
axs[1].set_title("Pre-processed template 1")
axs[2].set_title("Pre-processed template 2")

In [ ]:
input_images = [preprocess_image(image) for image in images_color]

fig, axs = plt.subplots(nrows=2, ncols=len(images_color))
fig.set_size_inches(25, 10)
for ax_color, ax_preprocess, image_color, input_image in zip(
    axs[0][:], axs[1][:], images_color, input_images
):
    ax_color.imshow(cv2.cvtColor(image_color, cv2.COLOR_BGR2RGB))
    ax_preprocess.imshow(input_image, cmap="gray")

### Aplicación de pirámides en el template
Se exploro su uso pero no es utilizado en el algoritmo final.

In [ ]:
# Get max width and height of the images to test
max_height = max([image.shape[0] for image in input_images])
max_width = max([image.shape[1] for image in input_images])
min_width = 70

# over-sampling
template_oversapling = [template1.copy()]
while (template_oversapling[-1].shape[0] < max_height) and (
    template_oversapling[-1].shape[1] < max_width
):
    template_oversapling.append(cv2.pyrUp(template_oversapling[-1]))

# sub-sampling
template_subsapling = [template1.copy()]
while template_subsapling[-1].shape[1] > min_width:
    template_subsapling.append(cv2.pyrDown(template_subsapling[-1]))

# to not repeat the original
template_pyramid = template_subsapling + template_oversapling[1:]

In [ ]:
print("NUMBER OF TEMPLATES:", len(template_pyramid))

fig, axs = plt.subplots(nrows=1, ncols=len(template_pyramid))
fig.set_size_inches(20, 5)
for ax, template in zip(axs, template_pyramid):
    ax.imshow(cv2.cvtColor(template, cv2.COLOR_BGR2RGB))

### Template matching
Se exploro su uso pero no es utilizado en el algoritmo final.

In [ ]:
metrics = [
    ("TM_CCOEFF", cv2.TM_CCOEFF),
    ("TM_CCOEFF_NORMED", cv2.TM_CCOEFF_NORMED),
    ("TM_CCORR", cv2.TM_CCORR),
    ("TM_CCORR_NORMED", cv2.TM_CCORR_NORMED),
    ("TM_SQDIFF", cv2.TM_SQDIFF),
    ("TM_SQDIFF_NORMED", cv2.TM_SQDIFF_NORMED),
]

fig, axs = plt.subplots(nrows=len(input_images), ncols=len(metrics))
fig.set_size_inches(20, 20)

for input_image_idx, (input_image, input_image_color) in enumerate(
    zip(input_images, images_color)
):
    for ax, (metric_name, metric_fn) in zip(axs[input_image_idx], metrics):
        image = input_image.copy()
        max_match_value = 0
        max_match = None
        for template in template_pyramid:
            if (template.shape[0] > image.shape[0]) or (
                template.shape[1] > image.shape[1]
            ):
                continue
            match = cv2.matchTemplate(image, template, metric_fn)
            min_value, max_value, min_location, max_location = cv2.minMaxLoc(match)
            if max_value > max_match_value:
                max_match = match

        min_value, max_value, min_location, max_location = cv2.minMaxLoc(max_match)
        if metric_name in ["TM_SQDIFF", "TM_SQDIFF_NORMED"]:
            top_left = min_location
        else:
            top_left = max_location
        bottom_right = (
            top_left[0] + template.shape[1],
            top_left[1] + template.shape[0],
        )

        image_color = input_image_color.copy()
        cv2.rectangle(image_color, top_left, bottom_right, 255, 2)
        ax.imshow(cv2.cvtColor(image_color, cv2.COLOR_BGR2RGB))
        ax.set_title(metric_name)

### Features matching con SIFT
Objetivo 1 cumplido: Obtener una detección del logo en cada imagen sin falsos positivos

In [ ]:
fig, axs = plt.subplots(nrows=len(input_images), ncols=1)
fig.set_size_inches(10, 20)

sift = cv2.SIFT_create()
bf_matcher = cv2.BFMatcher()

for input_image_idx, (ax, color_image, input_image) in enumerate(
    zip(axs, images_color, input_images)
):
    template = template2 if input_image_idx != 2 else template1
    keypoints_template, descriptors_template = sift.detectAndCompute(template, None)
    keypoints_input, descriptors_input = sift.detectAndCompute(input_image, None)
    matches = bf_matcher.knnMatch(descriptors_template, descriptors_input, k=2)

    good_matches = sorted(
        [m for m, n in matches if m.distance < 0.8 * n.distance],
        key=lambda x: x.distance,
    )
    template_points = np.float32(
        [keypoints_template[m.queryIdx].pt for m in good_matches]
    ).reshape(-1, 1, 2)
    input_points = np.float32(
        [keypoints_input[m.trainIdx].pt for m in good_matches]
    ).reshape(-1, 1, 2)

    homography, mask = cv2.findHomography(
        template_points, input_points, cv2.RANSAC, 3.0
    )
    draw_params = dict(
        matchColor=(0, 255, 255),
        singlePointColor=None,
        matchesMask=mask.ravel().tolist(),
        flags=2,
    )

    height, width = template.shape[:2]
    template_corners = np.float32(
        [[0, 0], [width, 0], [width, height], [0, height]]
    ).reshape(-1, 1, 2)
    transformed_corners = cv2.perspectiveTransform(template_corners, homography)

    plot_image = color_image.copy()
    plot_image = cv2.polylines(
        plot_image,
        [np.int32(transformed_corners)],
        isClosed=True,
        color=(0, 255, 0),
        thickness=3,
        lineType=cv2.LINE_AA,
    )
    plot_image = cv2.drawMatches(
        template,
        keypoints_template,
        plot_image,
        keypoints_input,
        good_matches,
        None,
        **draw_params
    )
    ax.imshow(cv2.cvtColor(plot_image, cv2.COLOR_BGR2RGB))

### Resources
- [Image Processing in OpenCV: Template matching](https://docs.opencv.org/4.x/d4/dc6/tutorial_py_template_matching.html)
- [Image Processing: Image Pyramids](https://docs.opencv.org/4.x/d4/d1f/tutorial_pyramids.html)
- [2D Features framework: Basic concepts of the homography explained with code](https://docs.opencv.org/4.x/d9/dab/tutorial_homography.html)